In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--disable-application-cache")
options.add_argument("--disable-cache")
options.add_argument("--disk-cache-size=0")

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
df = pd.read_csv("../datasets/urls.csv")
links = df["Url"]

In [34]:
filepath = "../datasets/specs.csv"

def update_specs(row):
        title = row['Title']
        value = row['Value']
        if title in specs:
            specs[title] = value

for x in range(5):
    titles = []
    values = []
    driver.get(links[x])
    print(x, links[x])

    driver.execute_script("window.scrollTo(0, 300)")
    try:
        view = driver.find_element(
            By.XPATH, '//*[@id="module_product_detail"]/div/div[2]/button'
        )
        driver.execute_script("arguments[0].scrollIntoView();", view)
        driver.execute_script("window.scrollBy(0, -100)")
        view.click()
        specs = driver.find_element(
            By.XPATH, '//*[@id="module_product_detail"]/div/div[1]/div[3]/h2'
        )
        driver.execute_script("arguments[0].scrollIntoView();", specs)
        driver.execute_script("window.scrollBy(0, -100)")
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    '//*[@id="module_product_detail"]/div/div/div[3]/div[1]/ul',
                )
            )
        )
        stitle = driver.find_elements(By.CLASS_NAME, "key-title")
        svalue = driver.find_elements(By.CLASS_NAME, "key-value")
    except:
        try:
            specs = driver.find_element(
                By.XPATH, '//*[@id="module_product_detail"]/div/div/div[3]/h2'
            )
            WebDriverWait(driver, 1).until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="module_product_detail"]/div/div/div[3]/div[1]/ul',
                    )
                )
            )
            stitle = driver.find_elements(By.CLASS_NAME, "key-title")
            svalue= driver.find_elements(By.CLASS_NAME, "key-value")
        except:
            specs = driver.find_element(
                By.XPATH, '//*[@id="module_product_detail"]/div/div/div[2]/h2'
            )
            WebDriverWait(driver, 1).until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="module_product_detail"]/div/div/div[2]/div[1]/ul',
                    )
                )
            )
            stitle = driver.find_elements(By.CLASS_NAME, "key-title")
            svalue= driver.find_elements(By.CLASS_NAME, "key-value")

    titles = [stitle[i].text for i in range(2, len(stitle) - 1)]
    values = [svalue[i].text for i in range(2, len(stitle) - 1)]
    temp = pd.DataFrame(list(zip(titles, values)), columns=["Title", "Value"])
    temp.loc[:, 'Title'] = temp['Title'].str.replace("_", " ")
    temp.loc[:, 'Title'] = temp['Title'].apply(lambda x: x.lower())
    temp = temp[(temp['Title'] != "camera front (megapixels)") & (temp['Title'] !="cpu speed (ghz)") & 
                            (temp['Title'] != "wireless connectivity") & 
                            (temp['Title'] != "input output ports") & 
                            (temp['Title'] != "battery life") &
                            (temp['Title'] != "ac adapter") &
                            (temp['Title'] != "model no.") &
                            (temp['Title'] != "model") &
                            (temp['Title'] != "generation") &
                            (temp['Title'] != "condition") &
                            (temp['Title'] != "storage type") &
                            (temp['Title'] != "processor type") &
                            (temp['Title'] != "touch pad")]
    temp.loc[:, 'Title'] = temp['Title'].str.replace("storage capacity new", "storage capacity")
    temp.loc[:, 'Title'] = temp['Title'].str.replace("number of cpu cores", "cpu cores")
    specs = {
        'processor': None, 
        'operating system': None,
        'ram memory': None, 
        'display size': None, 
        'storage capacity': None, 
        'cpu cores': None, 
        'graphics card': None, 
        'graphics memory': None, 
        'touch': None
    }
    df = pd.DataFrame(temp)
    df.apply(update_specs, axis=1)
    # df = pd.DataFrame([specs['processor'], specs['operating system'], specs['ram memory'], specs['display size'], specs['storage capacity'], specs['cpu cores'], specs['graphics card'], specs['graphics memory'], specs['touch']], columns=['processor', 'operating system', 'ram memory', 'display size', 'storage capacity', 'cpu cores', 'graphics card', 'graphics memory', 'touch'])
    dd = pd.DataFrame([specs])
    dd.to_csv(filepath, mode="a", header=not os.path.exists(filepath), index=False)


0 https://www.daraz.com.np/products/apple-macbook-air-13-inch-m1-256gb-oliz-store-i106600811-s1028165605.html?search=1
1 https://www.daraz.com.np/products/acer-nitro-5-ryzen-7-5800h-16gb-ram-512gb-ssd-gtx-1650-144hz-rgb-i128490145-s1035974947.html?search=1
2 https://www.daraz.com.np/products/hp-victus-15-fb1013dx-amd-ryzen-5-7535hs-processor-8gb-ram-512gb-storage-nvidia-rtx-2050-graphics-156-inch-fhd-144hz-display-i129862643-s1037713180.html?search=1
3 https://www.daraz.com.np/products/lenovo-v15-celeron-4gb-ram-256gb-ssd156-inch-hd-display-i117911554-s1032329631.html?search=1
4 https://www.daraz.com.np/products/hp-victus-15-core-i5-13th-gen-intel-13420h-nvidia-rtx-3050-6gb-graphics-16gb-ram512gb-ssd156-144hrzbacklit-keyboardwin-11-i128174010-s1035273454.html?search=1


In [ ]:

driver.delete_all_cookies()
driver.quit()
